In [374]:
import sys
import os

from flask import Flask, jsonify, request
from flask_cors import CORS
from apscheduler.schedulers.background import BackgroundScheduler
import yfinance as yf
import pandas as pd
import cot_reports as cot
from pandas.tseries.offsets import BDay
import base64
from utils.currencies_and_indexes import load_currency_data

In [375]:

data_cache_dir = "data_cache"
os.makedirs(data_cache_dir, exist_ok=True)

# Mapping for agricultural tickers
agri_cots_to_yf_tickers = {
    'WHEAT-SRW - CHICAGO BOARD OF TRADE': 'ZW=F',
    'WHEAT-HRW - CHICAGO BOARD OF TRADE':'ZW=F',
    'WHEAT-HRSpring - MINNEAPOLIS GRAIN EXCHANGE':'ZW=F',
    'CORN - CHICAGO BOARD OF TRADE': 'ZC=F',
    'OATS - CHICAGO BOARD OF TRADE': 'ZO=F',
    'SOYBEAN CSO - CHICAGO BOARD OF TRADE': 'ZS=F',
    'COTTON NO. 2 - ICE FUTURES U.S.': 'CT=F',
    'SUGAR NO. 11 - ICE FUTURES U.S.': 'SB=F',
    'COFFEE C - ICE FUTURES U.S.': 'KC=F',
}

# Commodity categories
gas_cots = [
    'EUR STYLE NATURAL GAS OPTIONS - NEW YORK MERCANTILE EXCHANGE',
    'HENRY HUB - NEW YORK MERCANTILE EXCHANGE',
    'HENRY HUB PENULTIMATE NAT GAS - NEW YORK MERCANTILE EXCHANGE'
]

agri_cots_list = [
    'COFFEE C - ICE FUTURES U.S.',
    'CORN - CHICAGO BOARD OF TRADE',
    'COTTON NO. 2 - ICE FUTURES U.S.',
    'OATS - CHICAGO BOARD OF TRADE',
    'SOYBEAN CSO - CHICAGO BOARD OF TRADE',
    'SUGAR NO. 11 - ICE FUTURES U.S.',
    'WHEAT-HRSpring - MINNEAPOLIS GRAIN EXCHANGE',
    'WHEAT-HRW - CHICAGO BOARD OF TRADE',
    'WHEAT-SRW - CHICAGO BOARD OF TRADE',
]

columns_to_drop = [
    "Open Interest (All)",
    "CFTC Contract Market Code", 
    "CFTC Market Code in Initials",
    "Open Interest (Old)",
    "Noncommercial Positions-Long (Old)",
    "Noncommercial Positions-Short (Old)",
    "Noncommercial Positions-Spreading (Old)",
    "Commercial Positions-Long (Old)",
    "Commercial Positions-Short (Old)",
    "Total Reportable Positions-Long (Old)",
    "Total Reportable Positions-Short (Old)",
    "Nonreportable Positions-Long (Old)",
    "Nonreportable Positions-Short (Old)",
    "Open Interest (Other)",
    "Noncommercial Positions-Long (Other)",
    "Noncommercial Positions-Short (Other)",
    "Noncommercial Positions-Spreading (Other)",
    "Commercial Positions-Long (Other)",
    "Commercial Positions-Short (Other)",
    "Total Reportable Positions-Long (Other)",
    "Total Reportable Positions-Short (Other)",	
    "Nonreportable Positions-Long (Other)",	
    "Nonreportable Positions-Short (Other)",	
    "Change in Open Interest (All)",	
    "Change in Noncommercial-Long (All)",	
    "Change in Noncommercial-Short (All)",	
    "Change in Noncommercial-Spreading (All)",	
    "Change in Commercial-Long (All)",	
    "Change in Commercial-Short (All)",	
    "Change in Total Reportable-Long (All)",	
    "Change in Total Reportable-Short (All)",	
    "Change in Nonreportable-Long (All)",	
    "Change in Nonreportable-Short (All)",	
    "% of Open Interest (OI) (All)",
    "% of Open Interest (OI) (Other)",	
    "% of OI-Noncommercial-Long (Other)",	
    "% of OI-Noncommercial-Short (Other)",	
    "% of OI-Noncommercial-Spreading (Other)",	
    "% of OI-Commercial-Long (Other)",	
    "% of OI-Commercial-Short (Other)",	
    "% of OI-Total Reportable-Long (Other)",	
    "% of OI-Total Reportable-Short (Other)",	
    "% of OI-Nonreportable-Long (Other)",
    "% of OI-Nonreportable-Short (Other)",
    "Traders-Total (Other)",
    "Traders-Noncommercial-Long (Other)",
    "Traders-Noncommercial-Short (Other)",
    "Traders-Noncommercial-Spreading (Other)",
    "Traders-Commercial-Long (Other)",
    "Traders-Commercial-Short (Other)",
    "Traders-Total Reportable-Long (Other)",
    "Traders-Total Reportable-Short (Other)",
    "Concentration-Gross LT =4 TDR-Long (Old)",
    "Concentration-Gross LT =4 TDR-Short (Old)",
    "Concentration-Gross LT =8 TDR-Long (Old)",
    "Concentration-Gross LT =8 TDR-Short (Old)",
    "Concentration-Net LT =4 TDR-Long (Old)",
    "Concentration-Net LT =4 TDR-Short (Old)",
    "Concentration-Net LT =8 TDR-Long (Old)",
    "Concentration-Net LT =8 TDR-Short (Old)",
    "Concentration-Gross LT =4 TDR-Long (Other)",
    "Concentration-Gross LT =4 TDR-Short(Other)",
    "Concentration-Gross LT =8 TDR-Long (Other)",
    "Concentration-Gross LT =8 TDR-Short(Other)",
    "Concentration-Net LT =4 TDR-Long (Other)",
    "Concentration-Net LT =4 TDR-Short (Other)",
    "Concentration-Net LT =8 TDR-Long (Other)",
    "Concentration-Net LT =8 TDR-Short (Other)",
    "Contract Units",
    "CFTC Contract Market Code (Quotes)",
    "CFTC Market Code in Initials (Quotes)",
    "CFTC Commodity Code (Quotes)",
    "CFTC Region Code",
    "CFTC Commodity Code",
    "% of Open Interest (OI)(Old)",	
    "% of OI-Noncommercial-Long (Old)",	
    "% of OI-Noncommercial-Short (Old)",	
    "% of OI-Noncommercial-Spreading (Old)",	
    "% of OI-Commercial-Long (Old)",	
    "% of OI-Commercial-Short (Old)",	
    "% of OI-Total Reportable-Long (Old)",	
    "% of OI-Total Reportable-Short (Old)",	
    "% of OI-Nonreportable-Long (Old)",	
    "% of OI-Nonreportable-Short (Old)",	
    'Traders-Total (All)',
    'Noncommercial Positions-Spreading (All)',
    '% of OI-Noncommercial-Spreading (All)',
    'Traders-Noncommercial-Spreading (All)',
    'Traders-Total (Old)',
    'Traders-Noncommercial-Long (Old)', 'Traders-Noncommercial-Short (Old)',
    'Traders-Noncommercial-Spreading (Old)',
    'Traders-Commercial-Long (Old)', 'Traders-Commercial-Short (Old)',
    'Traders-Total Reportable-Long (Old)',
    'Traders-Total Reportable-Short (Old)',
]

renaming_dict  ={
    'Market and Exchange Names': 'ticker',
    'As of Date in Form YYYY-MM-DD': 'date2',
    'Noncommercial Positions-Long (All)': 'Noncommercial Long',
    'Noncommercial Positions-Short (All)': 'Noncommercial Short',
    'Commercial Positions-Long (All)': 'Commercial Long',
    'Commercial Positions-Short (All)': 'Commercial Short',
    'Total Reportable Positions-Long (All)': 'Reportable Long',
    'Total Reportable Positions-Short (All)': 'Reportable Short',
    '% of OI-Noncommercial-Long (All)': '%OI-Noncommercial-Long',
    '% of OI-Noncommercial-Short (All)': '%OI-Noncommercial-Short',
    '% of OI-Commercial-Long (All)': '%OI-Commercial-Long',
    '% of OI-Commercial-Short (All)': '%OI-Commercial-Short',
    '% of OI-Total Reportable-Long (All)': '%OI-Reportable-Long',
    '% of OI-Total Reportable-Short (All)': '%OI-Reportable-Short',
    '% of OI-Nonreportable-Long (All)': '%OI-Nonreportable-Long',
    '% of OI-Nonreportable-Short (All)': '%OI-Nonreportable-Short',
    'Traders-Noncommercial-Long (All)': 'Traders-Noncommercial-Long',
    'Traders-Noncommercial-Short (All)': 'Traders-Noncommercial-Short',
    'Traders-Commercial-Long (All)': 'Traders-Commercial-Long',
    'Traders-Commercial-Short (All)': 'Traders-Commercial-Short',
    'Traders-Total Reportable-Long (All)': 'Traders-Total Reportable-Long',
    'Traders-Total Reportable-Short (All)': 'Traders-Total Reportable-Short',
    'Concentration-Gross LT = 4 TDR-Long (All)': 'Concentration 4 TDR-Long',
    'Concentration-Gross LT =4 TDR-Short (All)': 'Concentration 4 TDR-Short',
    'Concentration-Gross LT =8 TDR-Long (All)': 'Concentration 8 TDR-Long',
    'Concentration-Gross LT =8 TDR-Short (All)': 'Concentration 8 TDR-Short',
    'Concentration-Net LT =4 TDR-Long (All)': 'Concentration-Net 4 TDR-Long',
    'Concentration-Net LT =4 TDR-Short (All)': 'Concentration-Net 4 TDR-Short',
    'Concentration-Net LT =8 TDR-Long (All)': 'Concentration-Net 8 TDR-Long',
    'Concentration-Net LT =8 TDR-Short (All)': 'Concentration-Net 8 TDR-Short'
}


In [376]:
# works
def download_gas_prices():
    ticker = "NG=F"
    start_date = "2020-01-01"
    end_date = pd.Timestamp.now().strftime("%Y-%m-%d")
    gas_price = yf.download(ticker, start=start_date, end=end_date, interval="1d")
    gas_price = gas_price[['Close']]
    gas_price.index = pd.to_datetime(gas_price.index)
    return gas_price

In [379]:
gas = download_gas_prices()


*********************100%***********************]  1 of 1 completed

In [380]:

def download_agri_prices():
    """
    Download agricultural prices and combine them into a single DataFrame with clean column names.
    """
    unique_tickers = set(agri_cots_to_yf_tickers.values())
    all_prices = []  # Collect individual DataFrames
    start_date = "2020-01-01"
    end_date = pd.Timestamp.now().strftime("%Y-%m-%d")

    for yf_ticker in unique_tickers:
        print(f"Downloading price data for {yf_ticker}")
        try:
            price_data = yf.download(yf_ticker, start=start_date, end=end_date, interval="1d")
            if not price_data.empty:
                # Extract and rename 'Close' column
                price_data = price_data[['Close']].copy()
                price_data.index = pd.to_datetime(price_data.index)
                price_data.rename(columns={'Close': yf_ticker}, inplace=True)
                all_prices.append(price_data)
        except Exception as e:
            print(f"Failed to download data for {yf_ticker}: {e}")

    if all_prices:
        # Combine all prices into one DataFrame
        combined_prices = pd.concat(all_prices, axis=1).sort_index()

        # Fix column names to ensure they are clean
        combined_prices.columns = combined_prices.columns.get_level_values(0)
        
        return combined_prices.reset_index()
    else:
        return pd.DataFrame()  # Return empty DataFrame if no data is available
    

In [381]:
agri_prices = download_agri_prices()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

In [382]:
agri_prices

Price,Date,ZS=F,KC=F,SB=F,ZO=F,ZW=F,ZC=F,CT=F
0,2020-01-02,944.25,127.099998,13.130000,298.00,560.25,391.50,69.269997
1,2020-01-03,930.50,126.349998,13.310000,290.75,554.50,386.50,69.199997
2,2020-01-06,932.75,122.150002,13.730000,294.25,550.00,384.75,70.040001
3,2020-01-07,935.00,122.400002,13.590000,293.25,550.25,384.50,69.830002
4,2020-01-08,938.25,119.150002,13.470000,299.00,552.75,384.25,69.959999
...,...,...,...,...,...,...,...,...
1245,2024-12-11,995.50,321.700012,21.280001,334.75,543.50,438.00,70.150002
1246,2024-12-12,995.75,322.600006,20.889999,339.25,538.75,431.75,70.089996
1247,2024-12-13,988.25,320.850006,20.719999,340.50,526.50,430.00,69.269997
1248,2024-12-16,982.00,328.750000,20.680000,368.00,550.00,445.00,69.059998


In [383]:

def load_data():
    current_date_str = pd.Timestamp.now().strftime("%Y%m%d")
    agri_data_file = os.path.join(data_cache_dir, f"agri_data_{current_date_str}.h5")
    natgas_data_file = os.path.join(data_cache_dir, f"natgas_data_{current_date_str}.h5")

    if os.path.exists(agri_data_file) and os.path.exists(natgas_data_file):
        agri_multiindex = pd.read_hdf(agri_data_file, key='agri')
        gas_multiindex = pd.read_hdf(natgas_data_file, key='natgas')
        print("Loaded cached agricultural and natural gas data from HDF5 files.")
    else:
        # Load and preprocess COT data
        df = pd.concat([pd.DataFrame(cot.cot_year(i, cot_report_type='legacy_futopt')) for i in range(2020, 2025)], ignore_index=False)

        ### Agricultural Data Processing ###
        agri_df = df[df['Market and Exchange Names'].isin(agri_cots_list)].copy()
        agri_df.drop(columns=columns_to_drop, inplace=True)
        agri_df.rename(columns=renaming_dict, inplace=True)
        agri_df['date2'] = pd.to_datetime(agri_df['date2'])
        agri_multiindex = agri_df.set_index(['ticker', 'date2']).sort_index()

        # Download and align agricultural prices
        agri_prices = download_agri_prices()
        agri_prices['Date'] = pd.to_datetime(agri_prices['Date'])  # Ensure Date is datetime
        agri_prices.set_index('Date', inplace=True)  # Set index to Date

        # Map and populate prices into agri_multiindex
        agri_multiindex['YF_Price'] = agri_multiindex.apply(
            lambda row: agri_prices.loc[row.name[1], agri_cots_to_yf_tickers.get(row.name[0])]
            if row.name[1] in agri_prices.index and agri_cots_to_yf_tickers.get(row.name[0]) in agri_prices.columns else None,
            axis=1
        )

        agri_multiindex['Net Traders Noncommercial'] = agri_multiindex['Traders-Noncommercial-Long'] - agri_multiindex['Traders-Noncommercial-Short']
        agri_multiindex['Net Traders Commercial'] = agri_multiindex['Traders-Commercial-Long'] - agri_multiindex['Traders-Commercial-Short']
        agri_multiindex['Net Commercial'] = agri_multiindex['Commercial Long'] - agri_multiindex['Commercial Short']

        ### Gas Data Processing ###
        gas_df = df[df['Market and Exchange Names'].isin(gas_cots)].copy()
        gas_df.rename(columns={'Market and Exchange Names': 'ticker', 'As of Date in Form YYYY-MM-DD': 'date2'}, inplace=True)
        gas_df.drop(columns=columns_to_drop, inplace=True)
        gas_df.rename(columns=renaming_dict, inplace=True)
        gas_df['date2'] = pd.to_datetime(gas_df['date2'])

        gas_multiindex = gas_df.set_index(['ticker', 'date2']).sort_index()

        # Download and align gas prices
        gas_prices = download_gas_prices()
        gas_prices.index = pd.to_datetime(gas_prices.index)
        gas_prices = gas_prices.reindex(gas_prices.index, method='ffill')

        gas_multiindex['NG_Close'] = gas_multiindex.apply(
            lambda row: gas_prices.loc[row.name[1], 'Close']
            if row.name[1] in gas_prices.index else None,
            axis=1
        )
        gas_multiindex['Net Traders Noncommercial'] = gas_multiindex['Traders-Noncommercial-Long'] - gas_multiindex['Traders-Noncommercial-Short']
        gas_multiindex['Net Traders Commercial'] = gas_multiindex['Traders-Commercial-Long'] - gas_multiindex['Traders-Commercial-Short']
        gas_multiindex['Net Commercial'] = gas_multiindex['Commercial Long'] - gas_multiindex['Commercial Short']

        # Save processed data to cache
        agri_multiindex.to_hdf(agri_data_file, key='agri', mode='w')
        gas_multiindex.to_hdf(natgas_data_file, key='natgas', mode='w')

        print("Processed and saved agricultural and natural gas data to cache.")
        print("Agri MultiIndex Columns:", agri_multiindex.columns)
        print("Agri MultiIndex Index Levels:", agri_multiindex.index.names)

    return agri_multiindex, gas_multiindex




In [387]:
agri_multi, natgas_multi = load_data()

Selected: legacy_futopt
Downloaded single year data from: 2020
Stored the file annualof.txt in the working directory.
Selected: legacy_futopt
Downloaded single year data from: 2021
Stored the file annualof.txt in the working directory.
Selected: legacy_futopt
Downloaded single year data from: 2022
Stored the file annualof.txt in the working directory.
Selected: legacy_futopt
Downloaded single year data from: 2023
Stored the file annualof.txt in the working directory.
Selected: legacy_futopt


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Downloaded single year data from: 2024
Stored the file annualof.txt in the working directory.




*********************100%***********************]  1 of 1 completed

Processed and saved agricultural and natural gas data to cache.
Agri MultiIndex Columns: Index(['As of Date in Form YYMMDD', 'Noncommercial Long',
       'Noncommercial Short', 'Commercial Long', 'Commercial Short',
       ' Total Reportable Positions-Long (All)', 'Reportable Short',
       'Nonreportable Positions-Long (All)',
       'Nonreportable Positions-Short (All)', '%OI-Noncommercial-Long',
       '%OI-Noncommercial-Short', '%OI-Commercial-Long',
       '%OI-Commercial-Short', '%OI-Reportable-Long', '%OI-Reportable-Short',
       '%OI-Nonreportable-Long', '%OI-Nonreportable-Short',
       'Traders-Noncommercial-Long', 'Traders-Noncommercial-Short',
       'Traders-Commercial-Long', 'Traders-Commercial-Short',
       'Traders-Total Reportable-Long', 'Traders-Total Reportable-Short',
       'Concentration 4 TDR-Long', 'Concentration 4 TDR-Short',
       'Concentration 8 TDR-Long', 'Concentration 8 TDR-Short',
       'Concentration-Net 4 TDR-Long', 'Concentration-Net 4 TDR-Short',

In [388]:
agri_multi

As of Date in Form YYMMDD  \
ticker                             date2                                   
COFFEE C - ICE FUTURES U.S.        2020-01-07                     200107   
                                   2020-01-14                     200114   
                                   2020-01-21                     200121   
                                   2020-01-28                     200128   
                                   2020-02-04                     200204   
...                                                                  ...   
WHEAT-SRW - CHICAGO BOARD OF TRADE 2024-11-12                     241112   
                                   2024-11-19                     241119   
                                   2024-11-26                     241126   
                                   2024-12-03                     241203   
                                   2024-12-10                     241210   

                                               Noncommercial Long  \
ticker                             date2                            
COFFEE C - ICE FUTURES U.S.        2020-01-07               65482   
                                   2020-01-14               61125   
                                   2020-01-21               58514   
                                   2020-01-28               54773   
                                   2020-02-04               51857   
...                                                           ...   
WHEAT-SRW - CHICAGO BOARD OF TRADE 2024-11-12               96668   
                                   2024-11-19              105156   
                                   2024-11-26              110717   
                                   2024-12-03              116804   
                                   2024-12-10              117644   

                                               Noncommercial Short  \
ticker                             date2                             
COFFEE C - ICE FUTURES U.S.        2020-01-07                20098   
                                   2020-01-14                21515   
                                   2020-01-21                21650   
                                   2020-01-28                27642   
                                   2020-02-04                38302   
...                                                            ...   
WHEAT-SRW - CHICAGO BOARD OF TRADE 2024-11-12               140316   
                                   2024-11-19               157597   
                                   2024-11-26               170146   
                                   2024-12-03               185628   
                                   2024-12-10               184218   

                                               Commercial Long  \
ticker                             date2                         
COFFEE C - ICE FUTURES U.S.        2020-01-07           157601   
                                   2020-01-14           164414   
                                   2020-01-21           164435   
                                   2020-01-28           177644   
                                   2020-02-04           195452   
...                                                        ...   
WHEAT-SRW - CHICAGO BOARD OF TRADE 2024-11-12           196862   
                                   2024-11-19           197919   
                                   2024-11-26           170904   
                                   2024-12-03           178466   
                                   2024-12-10           185049   

                                               Commercial Short  \
ticker                             date2                          
COFFEE C - ICE FUTURES U.S.        2020-01-07            207086   
                                   2020-01-14            210100   
                                   2020-01-21            207488   
                                   2020-01-28          

In [389]:
natgas_multi

As of Date in Form YYMMDD  \
ticker                                             date2                                   
EUR STYLE NATURAL GAS OPTIONS - NEW YORK MERCAN... 2020-01-07                     200107   
                                                   2020-01-14                     200114   
                                                   2020-01-21                     200121   
                                                   2020-01-28                     200128   
                                                   2020-02-04                     200204   
...                                                                                  ...   
HENRY HUB PENULTIMATE NAT GAS - NEW YORK MERCAN... 2024-11-12                     241112   
                                                   2024-11-19                     241119   
                                                   2024-11-26                     241126   
                                                   2024-12-03                     241203   
                                                   2024-12-10                     241210   

                                                               Noncommercial Long  \
ticker                                             date2                            
EUR STYLE NATURAL GAS OPTIONS - NEW YORK MERCAN... 2020-01-07               37023   
                                                   2020-01-14               44118   
                                                   2020-01-21               56812   
                                                   2020-01-28               48513   
                                                   2020-02-04               52331   
...                                                                           ...   
HENRY HUB PENULTIMATE NAT GAS - NEW YORK MERCAN... 2024-11-12               54403   
                                                   2024-11-19               53493   
                                                   2024-11-26               54642   
                                                   2024-12-03               54980   
                                                   2024-12-10               55283   

                                                               Noncommercial Short  \
ticker                                             date2                             
EUR STYLE NATURAL GAS OPTIONS - NEW YORK MERCAN... 2020-01-07                15016   
                                                   2020-01-14                17731   
                                                   2020-01-21                22260   
                                                   2020-01-28                16735   
                                                   2020-02-04                17747   
...                                                                            ...   
HENRY HUB PENULTIMATE NAT GAS - NEW YORK MERCAN... 2024-11-12                42770   
                                                   2024-11-19                41164   
                                                   2024-11-26                42885   
                                                   2024-12-03                40996   
                                                   2024-12-10                39840   

                                                               Commercial Long  \
ticker                                             date2                         
EUR STYLE NATURAL GAS OPTIONS - NEW YORK MERCAN... 2020-01-07           267197   
                                                   2020-01-14           269951   
                                                   2020-01-21           314268   
                                                   2020-01-28           286438   
                                                   2020-02-04           304948   
...                                                                